In [24]:
%matplotlib widget
import casadi as ca
import matplotlib.pyplot as plt
import numpy as np

plt.close("all")

# Create an Opti instance
opti = ca.Opti()

# Solver options
opts = {
    #"qpsol": "ipopt",
    #"convexify_strategy": "regularize",
    "print_time": 0,
    #"print_iteration": 0,
    #"print_header": 0,
    "verbose": False,
    "error_on_fail": False
}

opti.solver('ipopt',opts)


#=========== Parameters of the static models
B1_power = np.array([[2.8481E-02,0,0],[0,3.3614E-02,0],[0,0,1.7562E-02]])
B2_power = np.array([[1.6230E-05,0,0],[0,5.1395E-05,0],[0,0,2.7402E-05]])
C_power = np.array([[7.4287E-02],[1.0755E+00],[7.6364E-02]])
zs_pwr = 0      # Zero shift power


B1_pressure = np.array([9.2727E-05])
B2_pressure = np.array([2.5979E-08])
C_pressure = np.array([5.3955E-01])
zs_pr = 0       # Zero shift power

#=========== Parameters of the ARX model
# array of params
A_outflow = np.array([-1.2312E-03,1.5751E-02,1.4445E-03,-1.4182E-02,1.7257E-03,-1.5083E-02,1.3046E-01,-9.8236E-04,1.8656E-02,1.9622E-02]).reshape((1,-1))
zs_oflw = 3     # Zero shift outflow
zs_w = 2     # Zero shift speed

#=========== Parameters of the ARX model
#A (ny x (na1+na2)) na1,2: lags on y1,2
# A_power = np.array([[8.6870E-01,0],[0,8.7838E-01]])
# A_outflow = np.array([[1.0869,-1.5832E-01]])

# B (ny x (nb1+nb2)) nb1,2: lags on u1,2
# B_power = np.array([[2.7560E-02,0,-2.2282E-02,0],[0,3.5775E-02,0,-3.0685E-02]])
# B_outflow = np.array([[4.2454E-01,6.6812E-01,-3.6354E-01,-3.6354E-01]])

# C_power = np.array([1.0790E-02,1.7240E-02])
# C_outflow = np.array([-1.5832E-01])

#=========== Desired value of y 
Qout_meas = opti.parameter(2)
Qin_est = opti.parameter()
w_meas =  opti.parameter(3,3)   #TODO: Remove hard-coded Dims
h_meas = opti.parameter()
E_meas = opti.parameter(3)
P_meas = opti.parameter()

#=========== Define the Horizon
N = 6

#=========== Declare Symbolic Variables (SISO but can be extended to MIMO or MISO)
Qout = opti.variable(N+zs_oflw)
Qin = opti.variable(N)
E = opti.variable(3,N+zs_pwr)
P = opti.variable(N+zs_pr)
w = opti.variable(3,N+zs_w)    # Inlcudes 2 historic values
h = opti.variable(N+1)
Qin[:] = Qin_est


#=========== Objective function
# Minimize the deviation of y from yd and control effort
objective = 0
for t in range(1, N):
    objective += 5*((E[0,t]+E[1,t]+E[2,t])/(Qout[t]+1))**2 + 0.1*(w[0,t]+w[1,t]+w[2,t])**2
  

opti.minimize(objective)

# ARX model constraints
for t in range(0, N):
    opti.subject_to(Qout[t+3] == A_outflow @ ca.vcat([Qout[t],w[2,t+2]**2, w[2,t+2]*Qout[t], w[2,t+2]*w[0,t+2],Qout[t]**2, w[2,t+2]*w[1,t+2], w[2,t+2], w[1,t+2]*w[0,t+2], w[1,t+2]**2, w[0,t+2]**2]))
    opti.subject_to(E[:,t] == B1_power @ w[:,t] + B2_power @ w[:,t]**2 + C_power)
    opti.subject_to(P[t] == B1_pressure @ Qout[t] + B2_pressure @ Qout[t]**2 + C_pressure)

Ts = 1      
A = 18

for t in range(0, N):    
    opti.subject_to(h[t+1] == h[t] + Ts/A*(Qin[t]-Qout[t]))

# Additional constraints (e.g., on control input)
for t in range(N):
    opti.subject_to(w[:,t] >= 0)  # Lower bound on control input
    opti.subject_to(w[:,t] <= 1500)   # Upper bound on control input
    opti.subject_to(h[t] >= 120)  # Lower bound on control input
    opti.subject_to(h[t] <= 200)   # Upper bound on control input
    opti.subject_to(P[t] <= 1)  # upper bound on pressure




# Initial conditions
opti.subject_to(Qout[0:2]   == Qout_meas)  # Initial value of y, read from sensor
opti.subject_to(h[0]        == h_meas)
opti.subject_to(w[:,0]      == w_meas[:,0])    #TODO: Doublecheck assignment here
opti.subject_to(w[:,1]      == w_meas[:,1])
opti.subject_to(w[:,2]      == w_meas[:,2])
opti.subject_to(E[:,0]      == E_meas[:,0])    #TODO: Doublecheck assignment here
# opti.subject_to(P[0]      == P_meas)
    



for i in range(3):   
    # Update the following values:
    # - Qin_est
    # - Qout_meas
    # - E_meas
    # - P_meas
    # - w_meas
    # - h_meas

    opti.set_value(Qin_est,50+i*100)
    opti.set_value(Qout_meas,[0,0])  # When set to [100,500] for testing: infeasible problem
    opti.set_value(h_meas,170)
    opti.set_value(w_meas,ca.hcat([[100,0,0],[500,0,0],[200,0,0]])) #Structure: [[u1(k-1), u2(k-1)],[u1(k-2) u2(k-2)]]  TODO: For some values: infeasible problem
    opti.set_value(E_meas,[0,0,0])
    opti.set_value(P_meas,0.5)


    sol = opti.solve();   # actual solve


plt.clf()    
ax1 = plt.subplot(1,1,1)
plt.plot(sol.value(w)[0,:],"r",label='w 1')
plt.plot(sol.value(w)[1,:],"g",label='w 2')
plt.ylabel('Speed [RPM]')
ax12 = ax1.twinx()
ax12.plot(sol.value(E)[0,:],"r--",label='Power 1')
ax12.plot(sol.value(E)[1,:],"g--",label='Power 2')
ax12.set_ylabel('Power [kW]')
ax12.tick_params(axis ='y', labelcolor = 'blue')
h1, l1 = ax1.get_legend_handles_labels()
h12, l12 = ax12.get_legend_handles_labels()
ax1.legend(h1+h12, l1+l12, loc=1) 
plt.show()

plt.figure()
plt.plot(sol.value(h),label='h')
plt.figure()
plt.plot(sol.value(Qin),label='Qin')
plt.plot(sol.value(Qout),label='Qout')
plt.legend()
plt.show()


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      147
Number of nonzeros in inequality constraint Jacobian.:       54
Number of nonzeros in Lagrangian Hessian.............:      102

Total number of variables............................:       64
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       51
Total number of inequality constraints...............:       54
        inequality constraints with only lower bounds:       24
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       30

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 5.00e+02 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:996: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'